In [ ]:
import torch
from torchvision import datasets, transforms
import numpy as np
from captum.attr import IntegratedGradients, Saliency, Deconvolution, InputXGradient, GuidedBackprop, NoiseTunnel, IntegratedGradients, Occlusion, GradientShap, FeatureAblation, FeaturePermutation
import os        

In [ ]:

attribution_methods = {
    'Saliency': Saliency,
    'NoiseTunnel_Saliency': lambda model: NoiseTunnel(Saliency(model)),
    'Deconvolution': Deconvolution,
    'InputXGradient': InputXGradient,
    'GuidedBackprop': GuidedBackprop,
    'GradientShap': GradientShap,
    'Occlusion': Occlusion,
    'IntegratedGradients': IntegratedGradients,
    'NoiseTunnel_Deconvolution': lambda model: NoiseTunnel(Deconvolution(model)),
    'NoiseTunnel_InputXGradient': lambda model: NoiseTunnel(InputXGradient(model)),
    'FeatureAblation': FeatureAblation,
    'FeaturePermutation': FeaturePermutation
}

In [ ]:

num_classes = 10
batch_size = 32
image_size = 224

device = torch.device('cpu')

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom sorting to ensure numerical order if filenames don't have leading zeros
test_dataset = datasets.ImageFolder(root=f'coco200_perclass', transform=transform)
test_dataset.samples.sort(key=lambda x: int(os.path.splitext(os.path.basename(x[0]))[0].replace('im', '')))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
from itertools import combinations

# List of models
models = ['resnet50', 'alexnet', 'convnext', 'vgg19', "resnet18", "vgg16", "vit", "vit_ssl", "resnet_ssl", "efficientnet", "swin"]

# Initialize L2 similarity dictionary
l2_similarities = {}

for method_name in attribution_methods.keys():
    print(method_name)
    l2_similarities[method_name] = {}
    
    for model1, model2 in combinations(models, 2):
        
        l2_similarities[method_name][(model1, model2)] = []
        
    for i in range(len(test_dataset)):
        # Load attribution maps for all models
        attr_maps = {
            model: np.load(f"./attribution_maps/{model}/{method_name}/image_{i}.npy") 
            for model in models
        }
        
        # Compute L2 similarities for all model pairs
        for model1, model2 in combinations(models, 2):
            epsilon = 1e-8  # Small positive value to prevent division by zero
            l2_sim = 1 / (np.linalg.norm(attr_maps[model1] - attr_maps[model2]) + epsilon)    

            l2_similarities[method_name][(model1, model2)].append(l2_sim)

In [ ]:
from six.moves import cPickle as pickle 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

save_dict(l2_similarities, './all_similarities.pkl')